In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import collections
from etils import epath
from ml_collections import config_dict
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import pandas as pd

from chirp import audio_utils
from chirp.inference import a2o_utils
from chirp.inference import interface
from chirp.inference import tf_examples
from chirp.inference import models
from chirp.models import metrics
from chirp.taxonomy import namespace
from chirp.inference.search import bootstrap
from chirp.inference.search import search
from chirp.inference.search import display
from chirp.inference.classify import classify
from chirp.inference.classify import data_lib
import requests
import json
import pickle
import sys
from scipy.io import wavfile
import os

2024-06-23 09:42:49.126471: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 09:42:49.126548: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 09:42:49.126568: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 09:42:49.206007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load embedding config
working_dir =  '.'
embeddings_path = epath.Path(working_dir) / 'embeddings_all'
labeled_data_path = epath.Path(working_dir) / 'labeled_outputs'
custom_classifier_path = epath.Path(working_dir) / 'custom_classifier'
bootstrap_config = bootstrap.BootstrapConfig.load_from_embedding_path(
      embeddings_path=embeddings_path,
      annotated_path=labeled_data_path)

project_state = bootstrap.BootstrapState(
    bootstrap_config, a2o_auth_token='')
window_s = bootstrap_config.model_config['window_size_s']
sample_rate = bootstrap_config.model_config['sample_rate']

2024-06-23 09:44:35.032799: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15381 MB memory:  -> device: 0, name: Quadro GP100, pci bus id: 0000:5b:00.0, compute capability: 6.0


In [3]:
# load embedding, train smaller model

merged = data_lib.MergedDataset.from_folder_of_folders(
    base_dir=labeled_data_path,
    embedding_model=project_state.embedding_model,
    time_pooling='mean',
    load_audio=False,
    target_sample_rate=sample_rate,
    audio_file_pattern='*',
    embedding_config_hash=bootstrap_config.embedding_config_hash(),
)

lbl_counts = np.sum(merged.data['label_hot'], axis=0)
print('num classes :', (lbl_counts > 0).sum())
print('mean ex / class :', lbl_counts.sum() / (lbl_counts > 0).sum())
print('min ex / class :', (lbl_counts + (lbl_counts == 0) * 1e6).min())

Embedding from Folder of Folders...
Checking for existing embeddings from Folder of Folders...
Found 0 existing embeddings.
Checking for new sources to embed from Folder of Folders...


0it [00:00, ?it/s]

2024-06-23 09:45:02.953672: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2b285eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-23 09:45:02.953736: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro GP100, Compute Capability 6.0
2024-06-23 09:45:04.133691: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-23 09:45:04.186901: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert
2024-06-23 09:45:08.781494: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-06-23 09:45:18.255793: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
1it [00:22, 22.48s/it]
1i


...embedded dataset in 46.81s...
    found 81 classes.
    class amedip_call / 0 : 1
    class amekes_call / 1 : 1
    class amerob_call / 2 : 14
    class amerob_song / 3 : 13
    class annhum_song / 4 : 2
    class bkbwoo_call / 5 : 14
    class bkhgro_call / 6 : 15
    class bkhgro_song / 7 : 13
    class brncre_call / 8 : 13
    class brncre_song / 9 : 22
    class btywar_song / 10 : 12
    class casfin_call / 11 : 6
    class casfin_song / 12 : 23
    class casvir_song / 13 : 31
    class chispa_song / 14 : 2
    class clanut_call / 15 : 6
    class comnig_call / 16 : 33
    class compoo_song / 17 : 36
    class comrav_call / 18 : 21
    class coohaw_call / 19 : 8
    class coohaw_song / 20 : 14
    class daejun_call / 21 : 22
    class daejun_song / 22 : 16
    class dusfly_call / 23 : 22
    class dusfly_song / 24 : 21
    class evegro_call / 25 : 26
    class flaowl_song / 26 : 25
    class foxspa_song / 27 : 9
    class gnttow_call / 28 : 29
    class gnttow_song / 29 : 11
  

In [4]:
train_ratio = 0.9 
train_examples_per_class = None  

num_seeds = 3

# Classifier training hyperparams.
# These should be good defaults.
batch_size = 32
num_epochs = 128
num_hiddens = -1
learning_rate = 1e-3

metrics = collections.defaultdict(list)
for seed in tqdm(range(num_seeds)):
  if num_hiddens > 0:
    model = classify.get_two_layer_model(
        num_hiddens, merged.embedding_dim, merged.num_classes, True)
  else:
    model = classify.get_linear_model(
        merged.embedding_dim, merged.num_classes)
  run_metrics = classify.train_embedding_model(
      model, merged, train_ratio, train_examples_per_class,
      num_epochs, seed, batch_size, learning_rate)
  metrics['acc'].append(run_metrics.top1_accuracy)
  metrics['auc_roc'].append(run_metrics.auc_roc)
  metrics['cmap'].append(run_metrics.cmap_value)
  metrics['maps'].append(run_metrics.class_maps)
  metrics['test_logits'].append(run_metrics.test_logits)

mean_acc = np.mean(metrics['acc'])
mean_auc = np.mean(metrics['auc_roc'])
mean_cmap = np.mean(metrics['cmap'])
# Merge the test_logits into a single array.
test_logits = {
    k: np.concatenate([logits[k] for logits in metrics['test_logits']])
    for k in metrics['test_logits'][0].keys()
}

print(f'acc:{mean_acc:5.2f}, auc_roc:{mean_auc:5.2f}, cmap:{mean_cmap:5.2f}')
for lbl, auc in zip(merged.labels, run_metrics.class_maps):
  if np.isnan(auc):
    continue
  print(f'\n{lbl:8s}, auc_roc:{auc:5.2f}')

  0%|          | 0/3 [00:00<?, ?it/s]2024-06-23 09:46:29.803124: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8981802564530550995
2024-06-23 09:46:29.803238: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6357226189113289516
2024-06-23 09:48:53.883768: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7458827095098341151
 33%|███▎      | 1/3 [02:33<05:06, 153.29s/it]2024-06-23 09:48:59.503618: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8981802564530550995
2024-06-23 09:48:59.503720: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6357226189113289516
2024-06-23 09:48:59.503760: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14969174540760464816
2024-06-23 09:48:59.50

acc: 0.93, auc_roc: 1.00, cmap: 0.83

amerob_call, auc_roc: 1.00

amerob_song, auc_roc: 1.00

bkbwoo_call, auc_roc: 1.00

bkhgro_call, auc_roc: 1.00

bkhgro_song, auc_roc: 1.00

brncre_call, auc_roc: 1.00

brncre_song, auc_roc: 1.00

btywar_song, auc_roc: 1.00

casfin_song, auc_roc: 1.00

casvir_song, auc_roc: 1.00

comnig_call, auc_roc: 1.00

compoo_song, auc_roc: 1.00

comrav_call, auc_roc: 1.00

coohaw_song, auc_roc: 1.00

daejun_call, auc_roc: 1.00

daejun_song, auc_roc: 1.00

dusfly_call, auc_roc: 1.00

dusfly_song, auc_roc: 1.00

evegro_call, auc_roc: 1.00

flaowl_song, auc_roc: 1.00

gnttow_call, auc_roc: 1.00

gnttow_song, auc_roc: 1.00

gockin_call, auc_roc: 1.00

gockin_song, auc_roc: 1.00

haiwoo_call, auc_roc: 1.00

haiwoo_song, auc_roc: 1.00

hamfly_song, auc_roc: 1.00

herthr_call, auc_roc: 1.00

herthr_song, auc_roc: 1.00

herwar_song, auc_roc: 1.00

houwre_song, auc_roc: 1.00

lazbun_song, auc_roc: 1.00

macwar_call, auc_roc: 0.99

macwar_song, auc_roc: 1.00

moublu_cal

In [5]:
# save model to outputs
model.save(epath.Path(working_dir) / epath.Path('outputs') / 'custom_classifier')

INFO:tensorflow:Assets written to: outputs/custom_classifier/assets


INFO:tensorflow:Assets written to: outputs/custom_classifier/assets


In [6]:
output_filepath = epath.Path(working_dir) / epath.Path('outputs_all') / 'inference_custom.csv'

class_thresholds = None # write all logits

include_classes = [] # if empty, includes all classes
exclude_classes = []

embeddings_ds = project_state.create_embeddings_dataset(
    shuffle_files=True)
classify.write_inference_csv(
    embeddings_ds=embeddings_ds,
    model=model,
    labels=merged.labels,
    output_filepath=output_filepath,
    threshold=class_thresholds,
    embedding_hop_size_s=bootstrap_config.embedding_hop_size_s,
    include_classes=include_classes,
    exclude_classes=exclude_classes)

87135it [1:11:04, 20.43it/s]





   Detection count:  1263169890
NonDetection count:  0


In [7]:
inference = pd.read_csv(output_filepath, skipinitialspace=True) # annoying initial space...

In [8]:
inference.to_parquet(epath.Path(working_dir) / epath.Path('outputs_all') / 'inference_custom.parquet')